## Actor Critic
---

* PG 의 장점 + DQN 의 장점을 취함
    * 두 방식의 결합이라고도 볼 수 있음
    * 안정적이면서 빠름
* PG 단점
    * 가중치로 G_t 를 사용하면 더 좋은 action 의 의미가 퇴색될 가능성 있음
    * M.C 기법으로 즉시 학습이 불가능하며 variance 가 높음
    * DQN 의 부트스트랩 속성과 낮은 variance 속성을 합침
* 수식을 통해 G_t 를 그냥 Q 또는 A 로 대체할 수 있음을 증명함
* value based 알고리즘이 사실 Q 평가만 했기 때문에 critic 이라는 이름을 붙임
* actor 네트워크, critic 네트워크 2 개 만듬
    * 이 2 개를 완전히 분리할 수도 있고, 일부를 공유하게 만들 수도 있음
* Advantage
    * Q-V 값
    * 각 action 이 평균에 비해 얼마나 좋은가?
    * 덜 좋은 action 은 그냥 안좋은 것 (평균보다 안좋은 양수값을 음수로 만듬)
    * 수치 자체가 적어지기 때문에 variance 도 줄어듬
    * 그렇다고 V 네트워크를 또 분리하진 않음
    * bellman optimality equation 을 쓰면 V 만으로 advantage 표현 가능
    * 어떤 action 을 하던 간에 가장 좋은 상태가 되는 V 만 고려하게 됨
    * 이제 이걸 Q 대신 log_pi 에 곱해주면 A2C 가 됨
    * Q 네트워크 분리한다고 off policy 되지 않음
    * critic 은 데이터 수집도, 액션 생성도 하지 않음
    * 오로지 actor 가 둘 다 수행하고 그래서 on policy 임
    * 근데 I.S 난 버퍼를 사용하면 off 가 됨 (ER, PER, HER)
* Asynchronous
    * 글로벌 A2C 네트워크를 비동기적으로 업데이트
    * 여러 개의 로컬 네트워크가 병렬적, 독립적으로 상호작용
    * exploration 에 유용하고 데이터 수집에 매우 유용함
    * 여러 개의 환경이 동시에 돌고 그 결과를 글로벌 네트워크에 전달하기 때문에 어떻게 보면 리플레이 버퍼와 유사하다고 볼 수 있음
    * 그래서 A3C 는 기본적으로 off policy
    * 그러나 자기 자신이 과거에 경험한 것과 여러 로컬들이 경험한 것의 미묘한 차이가 있음
    * 그러나 실제로 효과는 입증 불가? (A2C 도 충분히 좋다는 의견)
* off policy 가 좋은 이유?
    * exploration 의 문제임
    * 상호작용 문제, POMDP 환경에서 탐험은 성능과 일반화에 거의 필수
    * 또한 이미 수집된 데이터로 학습을 하므로 샘플 효율이 높음
    * 샘플 효율은 off line 으로 가면서 더 높아짐
    * 탐험은 결국 액션을 어떻게 선택하는가의 문제로 귀결됨
    * sample 효율측면에서 가장 두드러짐
* off policy 단점
    * 그러나 과적합의 위험이 있음
    * 또한 최적화하기 어려움 - 정책이 분리되어 있으니 분포 불일치라는 고질적인 문제
* deadly triad
    * 신경망 근사 + 부트스트랩 + off policy
    * 최적화 문제를 극한의 어려움으로 만듬
    * 주로 분포 불일치와 과도한 Q 과측정
    * 그러나 이런 점들은 트릭으로 완화 가능
    * 또 과측정은 유사한 state 에 방문했을 때 측정값이 그 근처 state 에서 덩달아 오르는 것이 일부 영향있음
* RL 딜레마
    * RL 은 지도학습과 다르게 답을 보고 배우는게 아니라 답을 찾는 학습방법임
    * 그래서 탐색을 최대한 해야함
    * 데이터가 이미지보다 엄청 많이 필요한데 그 데이터 수집이 이미지보다 더 어려움
    * 그래서 학습 자체가 어렵고 학습하더라도 일반화가 안 됨
    * 그러니까 다른 분야에서 해낸것 처럼 스케일업을 해야 함
    * DRL 이 되면서 ML 모델들의 장점, 특징과 약점도 함께 받아들임
* 편향 분산
    * ML 에서 편향 분산과 다름
    * RL 에서 편향이 크다는 것은 학습 한 후 실행할 때마다 잘못된 길로 계속 빠진다는 것
    * RL 에서 분산이 크다는 것은 학습 한 후 실행할 때마다 결과가 중구난방이란 것
    * 어쨌든 둘 다 일반화에 나쁜 상황임
    * DRL 은 ML 모델의 편향 분산 문제까지 다 가져와버림
    * 그러니까 DRL 은 더 문제가 많음
    * 안그래도 어려운 RL 에 모델까지 붙이니까 더 다루기 어려워짐
    * 일반화 달성과 차원의 저주를 벗어나기 위해서 모델을 도입했지만 그만큼 더 섬세한 제어가 필요
    * 차원의 저주는 차원이 늘어날때마다 필요한 학습에 데이터는 기하급수적 증가의 문제를 의미
* 편향 분산의 근본적 원인
    * MC 방법은 에피소드 단위로 업데이트 하는데 그 에피소드 마다 내용이 제 각각임
    * 그렇게 학습을 하니까 학습 이후에 돌릴 때마다 제각각임
    * TD 방법은 부트스트랩 하면서 이전 상태와 격차 (TD error) 를 줄이려고 함
    * 그러니까 일관성은 있는데 잘못된 길로 잘 빠질 수 있음
    * 어쨌든 둘 다 데이터를 늘리면 누그러듬
    * 대신 TD 방식이 스텝마다 학습하기 때문에 수렴이 더 빠름
* discrete vs continuous
    * 이 떡밥에 대해 너무 많이 작성했지만 또 차이점이 있음
    * continouous 되는 순간, Q (critic) 는 더 이상 state 만 받아 action 별 값을 출력하지 못함
    * 그래서 그 떄부터는 Q 는 state 와 action 을 모두 입력으로 받는 신경망이 되어야 함
    * 물론 discrete action 에서도 위처럼 표현할 수 있지만 state 만 받는게 더 효율적임
    * 물론 그 연속적 action 갯수는 여러 개가 될 수 있고 갯수 자체는 discrete 함
    * 그리고 state, action 을 모두 받기 때문에 네트워크가 어느정도 복잡도가 커지는 걸 감수해야 할 듯
* deterministic vs stochastic
    * 이건 또 다른 대형 떡밥이었고 이것도 또 논할게 남아 있음
    * 결정론적으로 변한 policy 는 action 의 확률성이 제거되어 버림
    * 그래서 policy 는 확률 분포가 아니라 결정 값이 됨
    * 대부분 이 땐 연속적 action 인 경우가 많음
* actor 와 critic 이 대부분의 가중치를 공유하고 head 만 따로 뺄 수 있음
    * 하지만 이 경우 불안정하게 될 가능성이 높음
    * 약간의 업데이트라도 네트워크 전체에 영향을 주므로
    * 간단한 환경에서만 사용하는게 좋을 듯
* critic 도 여러가지 선택권이 있음
    * continuous 이면 입력은 state, action 둘 다 줘야하지만 discrete 이면 state 만 입력으로 줘도 됨
    * Q 가 아니라 A, V 를 출력한다면 action 에 대한 고려는 안 해도 됨
    * 즉, 스칼라 값 하나만 출력하면 됨
* DPG 에 대한 추가 정보
    * action 에 대한 기댓값 계산이 빠지므로 더 적은 데이터로 잘 학습할 수 있는 data efficiency 가 높음
    * stochastic PG 보다 성능이 좋음
    * 기존의 방법들은 연속 action 에선 작동 안 됨
* memo
    * TD3 (twin delayed ddpg)
    * ddpg

In [ ]:
import argparse
import gym
import numpy as np
from itertools import count
from collections import namedtuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

# Cart Pole

parser = argparse.ArgumentParser(description='PyTorch actor-critic example')
parser.add_argument('--gamma', type=float, default=0.99, metavar='G',
                    help='discount factor (default: 0.99)')
parser.add_argument('--seed', type=int, default=543, metavar='N',
                    help='random seed (default: 543)')
parser.add_argument('--render', action='store_true',
                    help='render the environment')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='interval between training status logs (default: 10)')
args = parser.parse_args()


env = gym.make('CartPole-v1')
env.reset(seed=args.seed)
torch.manual_seed(args.seed)


SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])


class Policy(nn.Module):
    """
    implements both actor and critic in one model
    """
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)

        # actor's layer
        self.action_head = nn.Linear(128, 2)

        # critic's layer
        self.value_head = nn.Linear(128, 1)

        # action & reward buffer
        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        """
        forward of both actor and critic
        """
        x = F.relu(self.affine1(x))

        # actor: choses action to take from state s_t
        # by returning probability of each action
        action_prob = F.softmax(self.action_head(x), dim=-1)

        # critic: evaluates being in the state s_t
        state_values = self.value_head(x)

        # return values for both actor and critic as a tuple of 2 values:
        # 1. a list with the probability of each action over the action space
        # 2. the value from state s_t
        return action_prob, state_values


model = Policy()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
eps = np.finfo(np.float32).eps.item()


def select_action(state):
    state = torch.from_numpy(state).float()
    probs, state_value = model(state)

    # create a categorical distribution over the list of probabilities of actions
    m = Categorical(probs)

    # and sample an action using the distribution
    action = m.sample()

    # save to action buffer
    model.saved_actions.append(SavedAction(m.log_prob(action), state_value))

    # the action to take (left or right)
    return action.item()


def finish_episode():
    """
    Training code. Calculates actor and critic loss and performs backprop.
    """
    R = 0
    saved_actions = model.saved_actions
    policy_losses = [] # list to save actor (policy) loss
    value_losses = [] # list to save critic (value) loss
    returns = [] # list to save the true values

    # calculate the true value using rewards returned from the environment
    for r in model.rewards[::-1]:
        # calculate the discounted value
        R = r + args.gamma * R
        returns.insert(0, R)

    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)

    for (log_prob, value), R in zip(saved_actions, returns):
        advantage = R - value.item()

        # calculate actor (policy) loss
        policy_losses.append(-log_prob * advantage)

        # calculate critic (value) loss using L1 smooth loss
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([R])))

    # reset gradients
    optimizer.zero_grad()

    # sum up all the values of policy_losses and value_losses
    loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()

    # perform backprop
    loss.backward()
    optimizer.step()

    # reset rewards and action buffer
    del model.rewards[:]
    del model.saved_actions[:]


def main():
    running_reward = 10

    # run infinitely many episodes
    for i_episode in count(1):

        # reset environment and episode reward
        state, _ = env.reset()
        ep_reward = 0

        # for each episode, only run 9999 steps so that we don't
        # infinite loop while learning
        for t in range(1, 10000):

            # select action from policy
            action = select_action(state)

            # take the action
            state, reward, done, _, _ = env.step(action)

            if args.render:
                env.render()

            model.rewards.append(reward)
            ep_reward += reward
            if done:
                break

        # update cumulative reward
        running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward

        # perform backprop
        finish_episode()

        # log results
        if i_episode % args.log_interval == 0:
            print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
                  i_episode, ep_reward, running_reward))

        # check if we have "solved" the cart pole problem
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            break


if __name__ == '__main__':
    main()